In [1]:
import pandas as pd
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import Row
#from pyspark.sql.functions import *
from pyspark.ml.feature import *
import pickle
#import functions
from pyspark.sql import functions

from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DoubleType, LongType,FloatType
import warnings
warnings.filterwarnings("ignore")

In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark\\spark3'

In [3]:
input_uri="mongodb://127.0.0.1:27017/"
output_uri="mongodb://127.0.0.1:27017/"

In [4]:
spark=SparkSession.builder.appName("final_data").config("spark.mongodb.input.uri",input_uri) \
        .config("spark.mongodb.output.uri",output_uri) \
        .config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:2.4.2") \
        .getOrCreate()

### Loading data

In [5]:
data_schema = [StructField("NPI", IntegerType(),True),
               StructField("City", StringType(),True),
               StructField("State", StringType(),True),
               StructField("Speciality", StringType(),True),
               StructField("max_Tot_Drug_Cst", FloatType(),True),
               StructField("sum_Tot_Drug_Cst", FloatType(),True),
               StructField("avg_Tot_Drug_Cst", FloatType(),True),
               StructField("max_Total_claims", IntegerType(),True),
               StructField("sum_Total_claims", LongType(),True),
               StructField("avg_Total_claims", DoubleType(),True),
               StructField("max_Tot_Day_Suply", IntegerType(),True),
               StructField("sum_Tot_Day_Suply", LongType(),True),
               StructField("avg_Tot_Day_Suply", FloatType(),True),
               StructField("max_Tot_30day_Fills", FloatType(),True),
               StructField("sum_Tot_30day_Fills", FloatType(),True),
               StructField("avg_Tot_30day_Fills", FloatType(),True),
               StructField("First_name", StringType(),True),
               StructField("Last_name", StringType(),True),
               StructField("Total_payments", FloatType(),True)
              
              ]

In [6]:
final_schema = StructType(fields=data_schema)

In [ ]:
#df_drug_pay = spark.read.csv("../data/spark_csv/drug_payment.csv/",inferSchema=True,header=True)

In [11]:
#Final data

df_drug_pay = spark.read.csv("../data/spark_csv/drug_pay_final.csv",inferSchema=True,header=True)

In [12]:
df_drug_pay.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- NPI: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Speciality: string (nullable = true)
 |-- max_Tot_Drug_Cst: double (nullable = true)
 |-- sum_Tot_Drug_Cst: double (nullable = true)
 |-- avg_Tot_Drug_Cst: double (nullable = true)
 |-- max_Total_claims: integer (nullable = true)
 |-- sum_Total_claims: integer (nullable = true)
 |-- avg_Total_claims: double (nullable = true)
 |-- max_Tot_Day_Suply: integer (nullable = true)
 |-- sum_Tot_Day_Suply: integer (nullable = true)
 |-- avg_Tot_Day_Suply: double (nullable = true)
 |-- max_Tot_30day_Fills: double (nullable = true)
 |-- sum_Tot_30day_Fills: double (nullable = true)
 |-- avg_Tot_30day_Fills: double (nullable = true)
 |-- First_name: string (nullable = true)
 |-- Last_name: string (nullable = true)
 |-- Total_payments: double (nullable = true)



In [13]:
df_drug_pay.show(5)

+---+----------+---------+-----+--------------------+----------------+----------------+----------------+----------------+----------------+------------------+-----------------+-----------------+-----------------+-------------------+-------------------+-------------------+----------+---------+--------------+
|_c0|       NPI|     City|State|          Speciality|max_Tot_Drug_Cst|sum_Tot_Drug_Cst|avg_Tot_Drug_Cst|max_Total_claims|sum_Total_claims|  avg_Total_claims|max_Tot_Day_Suply|sum_Tot_Day_Suply|avg_Tot_Day_Suply|max_Tot_30day_Fills|sum_Tot_30day_Fills|avg_Tot_30day_Fills|First_name|Last_name|Total_payments|
+---+----------+---------+-----+--------------------+----------------+----------------+----------------+----------------+----------------+------------------+-----------------+-----------------+-----------------+-------------------+-------------------+-------------------+----------+---------+--------------+
|  0|1003000126| bethesda|   md|   Internal Medicine|         4792.85|      

In [14]:
df_drug_pay = df_drug_pay.drop("_c0")

In [15]:
df_drug_pay.printSchema()

root
 |-- NPI: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Speciality: string (nullable = true)
 |-- max_Tot_Drug_Cst: double (nullable = true)
 |-- sum_Tot_Drug_Cst: double (nullable = true)
 |-- avg_Tot_Drug_Cst: double (nullable = true)
 |-- max_Total_claims: integer (nullable = true)
 |-- sum_Total_claims: integer (nullable = true)
 |-- avg_Total_claims: double (nullable = true)
 |-- max_Tot_Day_Suply: integer (nullable = true)
 |-- sum_Tot_Day_Suply: integer (nullable = true)
 |-- avg_Tot_Day_Suply: double (nullable = true)
 |-- max_Tot_30day_Fills: double (nullable = true)
 |-- sum_Tot_30day_Fills: double (nullable = true)
 |-- avg_Tot_30day_Fills: double (nullable = true)
 |-- First_name: string (nullable = true)
 |-- Last_name: string (nullable = true)
 |-- Total_payments: double (nullable = true)



### LELE data

In [16]:
df_fraud_1 = spark.read.csv("../data/UPDATED.csv",inferSchema=True,header=True)

In [17]:
df_fraud_1.printSchema()

root
 |-- LASTNAME: string (nullable = true)
 |-- FIRSTNAME: string (nullable = true)
 |-- MIDNAME: string (nullable = true)
 |-- BUSNAME: string (nullable = true)
 |-- GENERAL: string (nullable = true)
 |-- SPECIALTY: string (nullable = true)
 |-- UPIN: string (nullable = true)
 |-- NPI: integer (nullable = true)
 |-- DOB: integer (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- ZIP: string (nullable = true)
 |-- EXCLTYPE: string (nullable = true)
 |-- EXCLDATE: string (nullable = true)
 |-- REINDATE: integer (nullable = true)
 |-- WAIVERDATE: integer (nullable = true)
 |-- WVRSTATE: string (nullable = true)



### Data preprocessing

### Making dataframe with firstname,lastname and speciality

In [145]:
df_test = df_fraud_1.toPandas()

In [146]:
df_test[df_test.NPI != 0].count()

LASTNAME      5891
FIRSTNAME     5891
MIDNAME       6310
BUSNAME        419
GENERAL       6310
SPECIALTY     6191
UPIN          1448
NPI           6310
DOB           5891
ADDRESS       6310
CITY          6310
STATE         6309
ZIP           6310
EXCLTYPE      6310
EXCLDATE      6310
REINDATE      6310
WAIVERDATE    6310
WVRSTATE         6
dtype: int64

In [147]:
df_test.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
0,None,None,,"#1 MARKETING SERVICE, INC",OTHER BUSINESS,SOBER HOME,None,0,NaN,239 BRIGHTON BEACH AVENUE,BROOKLYN,NY,11235,1128a1,20200319,0,0,None
1,None,None,,101 FIRST CARE PHARMACY INC,OTHER BUSINESS,PHARMACY,None,1972902351,NaN,"C/O 609 W 191ST STREET, APT D",NEW YORK,NY,10040,1128b8,20220320,0,0,None
2,None,None,,14 LAWRENCE AVE PHARMACY,PHARMACY,None,None,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,None
3,None,None,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,None,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,None
4,None,None,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,None,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,None


In [148]:
df_test["FIRSTNAME"].fillna("Empty",inplace=True)

In [149]:
df_test["LASTNAME"].fillna("Empty",inplace=True)

In [150]:
df_test["SPECIALTY"].fillna("Empty",inplace=True)

In [151]:
df_test["CITY"].fillna("Not_filled",inplace=True)

In [152]:
df_test.isna().sum()

LASTNAME          0
FIRSTNAME         0
MIDNAME           0
BUSNAME       72895
GENERAL           0
SPECIALTY         0
UPIN          69928
NPI               0
DOB            4095
ADDRESS           9
CITY              0
STATE             5
ZIP               0
EXCLTYPE          0
EXCLDATE          0
REINDATE          0
WAIVERDATE        0
WVRSTATE      76107
dtype: int64

In [153]:
#df_test[(df_test.FIRSTNAME != "Empty") & (df_test.SPECIAL != "Empty") ]

In [154]:
#df_test[(df_test.FIRSTNAME != "Empty")  & (df_test.LASTNAME != "Empty")]

In [155]:
#df_second = df_test[(df_test.FIRSTNAME != "Empty") & (df_test.LASTNAME != "Empty") & (df_test.SPECIALTY != "Empty") & (df_test.NPI != 0)].loc[: ,["FIRSTNAME","LASTNAME","SPECIALTY"]]

In [156]:
#df_second.head()

In [157]:
#df_second.count()

In [158]:
df_second = df_test.copy()

In [159]:
df_second.head(2)

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
0,Empty,Empty,,"#1 MARKETING SERVICE, INC",OTHER BUSINESS,SOBER HOME,None,0,NaN,239 BRIGHTON BEACH AVENUE,BROOKLYN,NY,11235,1128a1,20200319,0,0,None
1,Empty,Empty,,101 FIRST CARE PHARMACY INC,OTHER BUSINESS,PHARMACY,None,1972902351,NaN,"C/O 609 W 191ST STREET, APT D",NEW YORK,NY,10040,1128b8,20220320,0,0,None


In [160]:
df_second["Name_2"] = df_second.FIRSTNAME+" " + df_second.LASTNAME

In [161]:
df_second["Name_2"] = df_second["Name_2"].str.lower()

In [162]:
df_second["SPECIALTY"] = df_second["SPECIALTY"].str.lower()

In [163]:
df_second["FIRSTNAME"] = df_second["FIRSTNAME"].str.lower()

In [164]:
df_second["LASTNAME"] = df_second["LASTNAME"].str.lower()

In [165]:
df_second["CITY"] = df_second["CITY"].str.lower()

In [166]:
#df_second.drop(["FIRSTNAME","LASTNAME"],axis =1 , inplace = True)

In [167]:
df_second["fraud_2"] = 1

In [168]:
df_second[df_second["FIRSTNAME"] != "empty"]

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,Name_2,fraud_2
3230,aalders,nancy,BLOCK,None,HOSPITAL,nurse/nurses aide,None,0,19640305.0,9527 SOUTH KENTON,oak lawn,IL,60453,1128b4,20030820,0,0,None,nancy aalders,1
3231,aaltonen,nickolas,A,None,IND- LIC HC SERV PRO,pharmacy technician,None,0,19880123.0,32 W PACIFIC AVE,spokane,WA,99201,1128a4,20120419,0,0,None,nickolas aaltonen,1
3232,aamir,muhammad,SOHAIL,None,BUS OWNER/EXEC,home health agency,None,0,19700910.0,3956 COLUMBIA COURT,bloomfield hills,MI,48302,1128a1,20170220,0,0,None,muhammad aamir,1
3233,aaron,alina,A,None,IND- LIC HC SERV PRO,nurse/nurses aide,None,0,19870209.0,3325 N ROSE CIRCLE DRIVE,phoenix,AZ,85018,1128b4,20171019,0,0,None,alina aaron,1
3234,aaron,christopher,WILLIAM,None,IND- LIC HC SERV PRO,nurse/nurses aide,None,0,19870908.0,120 SUPERVISOR DRIVE,west newton,PA,15089,1128b4,20140520,0,0,None,christopher aaron,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76120,zwolinski,amber,DAWN,None,IND- LIC HC SERV PRO,nurse/nurses aide,None,0,19870529.0,1238 PULASKI AVENUE,coal township,PA,17866,1128b4,20111220,0,0,None,amber zwolinski,1
76121,zyderveld,mary,ANN,None,PHARMACY,technician,None,0,19520825.0,1358 MORNING GLORY TURN,ruckersville,VA,22962,1128b4,20090219,0,0,None,mary zyderveld,1
76122,zylstra,christeena,LYNN,None,NURSING PROFESSION,nurse/nurses aide,None,0,19670724.0,1816 NE 49TH STREET,kansas city,MO,64118,1128b4,20100318,0,0,None,christeena zylstra,1
76123,zylstra,james,PETER,None,REHAB FACILITY - GEN,employee,None,0,19550420.0,"2305 MINNESOTA BLVD, S E",st cloud,MN,56304,1128a2,20040920,0,0,None,james zylstra,1


### Making dataframe with NPI numbers

In [ ]:
df_fraud_1.na.fill({"NPI":0})

In [ ]:
df_fraud_npi = df_fraud_1.filter(df_fraud_1["NPI"] != 0).select("NPI")

In [ ]:
#6310

df_fraud_npi.count()

In [ ]:
#Fraud NPI

df_fraud_npi.show()

In [ ]:
df_fraud_value = df_fraud_npi.withColumn("fraud", functions.lit(1))

In [ ]:
df_fraud_value.show()

In [ ]:
df_fraud_value =  df_fraud_value.withColumnRenamed("NPI","NPI_fraud")

In [ ]:
df_fraud_value.printSchema()

### Lets merge both dataframes i.e drug_pay and fraud_value

In [ ]:
df_final = df_drug_pay.join(df_fraud_value,df_drug_pay.NPI == df_fraud_value.NPI_fraud,"left")


In [ ]:
#We have null values in fraud

df_final.printSchema()

In [ ]:
df_final = df_final.drop("NPI_fraud")

In [ ]:
#Filling null values in fraud with 0 , indicating non fraud

df_final = df_final.na.fill({"fraud": 0})

In [ ]:
#Fraud column has zero nulls as nullable = false

df_final.printSchema()

In [ ]:
#169 frauds with NPI left join

df_final.groupBy('fraud').count().orderBy('count').show()

### Adding fraud data as name and specialty as criteria

In [195]:
df_finalll = df_final.toPandas()

In [196]:
df_finalll.head()

,NPI,City,State,Speciality,max_Tot_Drug_Cst,sum_Tot_Drug_Cst,avg_Tot_Drug_Cst,max_Total_claims,sum_Total_claims,avg_Total_claims,max_Tot_Day_Suply,sum_Tot_Day_Suply,avg_Tot_Day_Suply,max_Tot_30day_Fills,sum_Tot_30day_Fills,avg_Tot_30day_Fills,First_name,Last_name,Total_payments,fraud
0,1003000126,bethesda,md,Internal Medicine,4792.85,5979.07,747.38370,30,124,15.500000,937,3721,465.1250,32.4,138.2,17.275000,ardalan,enkeshafi,20.48,0
1,1003000423,cleveland,oh,Obstetrics & Gynecology,8276.44,15389.69,2564.94820,39,122,20.333333,2412,5217,869.5000,82.0,213.7,35.616665,jennifer,velotta,171.29,0
2,1003000720,clemmons,nc,Nurse Practitioner,657.01,3052.84,254.40334,51,230,19.166667,337,2166,180.5000,51.0,230.0,19.166666,None,None,NaN,0
3,1003001785,tulsa,ok,Orthopedic Surgery,2141.24,7812.28,781.22800,126,546,54.600000,2610,8250,825.0000,126.0,569.0,56.900000,jaclyn,jones,969.07,0
4,1003001884,flint,mi,Family Practice,1324.26,2449.09,349.87000,31,118,16.857143,1560,4979,711.2857,53.0,199.0,28.428572,None,None,NaN,0


In [197]:
df_finalll["Speciality"] = df_finalll["Speciality"].str.lower()

In [199]:
df_finalll = pd.merge(df_finalll, df_second.loc[:,['FIRSTNAME',"LASTNAME",'SPECIALTY','CITY',"fraud_2"]], how="left", left_on=['First_name',"Last_name", 'Speciality',"City"], right_on=['FIRSTNAME',"LASTNAME", 'SPECIALTY','CITY'])

In [ ]:
df_finalll.drop(["SPECIALTY",'FIRSTNAME',"LASTNAME",'CITY'],axis=1,inplace =True)

In [198]:
#df_finalll = pd.merge(df_finalll, df_second.loc[:,['FIRSTNAME','SPECIALTY','CITY',"fraud_2"]], how="left", left_on='First_name', right_on='FIRSTNAME')

In [200]:
#df_finalll = pd.merge(df_finalll, df_second.loc[:,['FIRSTNAME','SPECIALTY','CITY',"fraud_2"]], how="left", left_on=['First_name', 'Speciality',"City"], right_on=['FIRSTNAME', 'SPECIALTY','CITY'])

In [208]:
#df_finalll.drop(["SPECIALTY",'FIRSTNAME','CITY'],axis=1,inplace =True)

In [209]:
df_finalll.head()

,NPI,City,State,Speciality,max_Tot_Drug_Cst,sum_Tot_Drug_Cst,avg_Tot_Drug_Cst,max_Total_claims,sum_Total_claims,avg_Total_claims,...,sum_Tot_Day_Suply,avg_Tot_Day_Suply,max_Tot_30day_Fills,sum_Tot_30day_Fills,avg_Tot_30day_Fills,First_name,Last_name,Total_payments,fraud,fraud_2
0,1003000126,bethesda,md,internal medicine,4792.85,5979.07,747.38370,30,124,15.500000,...,3721,465.1250,32.4,138.2,17.275000,ardalan,enkeshafi,20.48,0,0
1,1003000423,cleveland,oh,obstetrics & gynecology,8276.44,15389.69,2564.94820,39,122,20.333333,...,5217,869.5000,82.0,213.7,35.616665,jennifer,velotta,171.29,0,0
2,1003000720,clemmons,nc,nurse practitioner,657.01,3052.84,254.40334,51,230,19.166667,...,2166,180.5000,51.0,230.0,19.166666,None,None,NaN,0,0
3,1003001785,tulsa,ok,orthopedic surgery,2141.24,7812.28,781.22800,126,546,54.600000,...,8250,825.0000,126.0,569.0,56.900000,jaclyn,jones,969.07,0,0
4,1003001884,flint,mi,family practice,1324.26,2449.09,349.87000,31,118,16.857143,...,4979,711.2857,53.0,199.0,28.428572,None,None,NaN,0,0


In [203]:
#Filling NA values with 0

df_finalll["fraud_2"].fillna(0,inplace =True)

In [204]:
df_finalll["fraud_2"] = df_finalll["fraud_2"].astype("int")

In [205]:
df_finalll.FIRSTNAME.value_counts().sum()

726

In [206]:
df_finalll.fraud_2.value_counts()

0    964330
1       726
Name: fraud_2, dtype: int64

In [210]:
#Coparing last two columns of dataframe : fraud_1 and fraud_2

def logic(data):
    content= []
    for i in data.values:
        #if both are zero
        if (i[-1] == 0) & (i[-2] == 0):
            content.append(0)
        #if any one is non zero
        elif (i[-1] == 0) & (i[-2] == 1):
            content.append(1)
        elif (i[-1] == 0) & (i[-2] == 1):
            content.append(1)
        else:
            pass
            
    return content

In [213]:
df_finalll["FRAUD"] = logic(df_finalll)

In [214]:
df_finalll.FRAUD.value_counts()

0    964170
1       886
Name: FRAUD, dtype: int64

In [215]:
df_finalll.drop(["fraud","fraud_2"],axis =1, inplace = True)

In [216]:
df_finalll.head(2)

,NPI,City,State,Speciality,max_Tot_Drug_Cst,sum_Tot_Drug_Cst,avg_Tot_Drug_Cst,max_Total_claims,sum_Total_claims,avg_Total_claims,max_Tot_Day_Suply,sum_Tot_Day_Suply,avg_Tot_Day_Suply,max_Tot_30day_Fills,sum_Tot_30day_Fills,avg_Tot_30day_Fills,First_name,Last_name,Total_payments,FRAUD
0,1003000126,bethesda,md,internal medicine,4792.85,5979.07,747.3837,30,124,15.500000,937,3721,465.125,32.4,138.2,17.275000,ardalan,enkeshafi,20.48,0
1,1003000423,cleveland,oh,obstetrics & gynecology,8276.44,15389.69,2564.9482,39,122,20.333333,2412,5217,869.500,82.0,213.7,35.616665,jennifer,velotta,171.29,0


In [217]:
df_finalll.isnull().sum()

NPI                         0
City                        0
State                       0
Speciality                  1
max_Tot_Drug_Cst            0
sum_Tot_Drug_Cst            0
avg_Tot_Drug_Cst            0
max_Total_claims            0
sum_Total_claims            0
avg_Total_claims            0
max_Tot_Day_Suply           0
sum_Tot_Day_Suply           0
avg_Tot_Day_Suply           0
max_Tot_30day_Fills         0
sum_Tot_30day_Fills         0
avg_Tot_30day_Fills         0
First_name             624632
Last_name              624632
Total_payments         624634
FRAUD                       0
dtype: int64

In [225]:
df_finalll.drop(["First_name","Last_name"],axis =1, inplace = True)

In [218]:
df_finalll.FRAUD.value_counts()

0    964170
1       886
Name: FRAUD, dtype: int64

### Saving dataframe

In [ ]:
final_df = spark.createDataFrame(df_finalll)


In [ ]:
final_df.printSchema()

root
 |-- NPI: long (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Speciality: string (nullable = true)
 |-- max_Tot_Drug_Cst: double (nullable = true)
 |-- sum_Tot_Drug_Cst: double (nullable = true)
 |-- avg_Tot_Drug_Cst: double (nullable = true)
 |-- max_Total_claims: long (nullable = true)
 |-- sum_Total_claims: long (nullable = true)
 |-- avg_Total_claims: double (nullable = true)
 |-- max_Tot_Day_Suply: long (nullable = true)
 |-- sum_Tot_Day_Suply: long (nullable = true)
 |-- avg_Tot_Day_Suply: double (nullable = true)
 |-- max_Tot_30day_Fills: double (nullable = true)
 |-- sum_Tot_30day_Fills: double (nullable = true)
 |-- avg_Tot_30day_Fills: double (nullable = true)
 |-- Total_payments: double (nullable = true)
 |-- FRAUD: long (nullable = true)



In [ ]:
final_df.write.format("com.mongodb.spark.sql.DefaultSource") \
    .option("database","Medicare") \
    .option("collection","final_dataframe") \
    .mode("overwrite").save()

In [ ]:
#Second option of loading data in to mongo db

# client = MongoClient('localhost',27017)
# db = client["Medicare"]
# collection = db["final_dataframe"]
# collection.insert_many(df_finall1.apply(lambda x: x.to_dict(), axis=1).to_list())

In [ ]:
dfraud = spark.createDataFrame(df_fraud_1)

In [11]:
dffraud.write.format("com.mongodb.spark.sql.DefaultSource") \
    .option("database","Medicare") \
    .option("collection","fraud_data") \
    .mode("overwrite").save()

In [228]:
df_finalll.to_csv("../data/spark_csv/final_dataframe.csv")